In [ ]:
import pandas as pd
data = pd.read_csv('merged_ogi_agg.csv', error_bad_lines=False);
data_text = data[['facts']]
data_text = data_text.dropna()
data_text['index'] = data_text.index
documents = data_text

In [ ]:
print(len(documents))
print(documents[:15])

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

In [ ]:
f = open('stopwords-zh.txt', 'r', encoding='UTF8')
stopwords = f.read().split('\n')
stopwords.append('ldquo')
stopwords.append('rdquo')
#stopwords

In [ ]:
# def lemmatize_stemming(text):
#     stemmer = SnowballStemmer("english")
#     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stopwords:
            result.append(token)
    return result

In [ ]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

In [ ]:
processed_docs = documents['facts'].map(preprocess)
processed_docs[:10]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=7, no_above=0.5, keep_n=7000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

In [ ]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
data_text['facts'][4310]

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

In [ ]:
wincodes = data[data['win_code'] == 1]
losecodes = data[data['lose_code'] == 1]

In [ ]:
win_text = wincodes[['facts']]
win_text = win_text.dropna()
win_text['index'] = win_text.index
win_documents = win_text

In [ ]:
lose_text = losecodes[['facts']]
lose_text = lose_text.dropna()
lose_text['index'] = lose_text.index
lose_documents = lose_text

In [ ]:
win_processed_docs = win_documents['facts'].map(preprocess)
win_processed_docs[:10]

In [ ]:
lose_processed_docs = lose_documents['facts'].map(preprocess)
lose_processed_docs[:10]

In [ ]:
#start BOW for win_code
win_dict = gensim.corpora.Dictionary(win_processed_docs)
count = 0
for k, v in win_dict.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
win_dict.filter_extremes(no_below=7, no_above=0.5, keep_n=7000)

In [ ]:
bow_win_corpus = [win_dict.doc2bow(doc) for doc in win_processed_docs]

In [ ]:
#tfidf for win_code
from gensim import corpora, models
tfidf = models.TfidfModel(bow_win_corpus)
win_corpus_tfidf = tfidf[bow_win_corpus]
from pprint import pprint
for doc in win_corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_win_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(win_corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
#5 topics for win_code
lda_model_tfidf = gensim.models.LdaMulticore(win_corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
#start BOW for lose_code
lose_dict = gensim.corpora.Dictionary(lose_processed_docs)
count = 0
for k, v in lose_dict.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
lose_dict.filter_extremes(no_below=7, no_above=0.5, keep_n=7000)

In [ ]:
bow_lose_corpus = [lose_dict.doc2bow(doc) for doc in lose_processed_docs]

In [ ]:
#tfidf for lose_code
from gensim import corpora, models
tfidf = models.TfidfModel(bow_lose_corpus)
lose_corpus_tfidf = tfidf[bow_lose_corpus]
from pprint import pprint
for doc in lose_corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_lose_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(lose_corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
#5 topics for lose_code
lda_model_tfidf = gensim.models.LdaMulticore(lose_corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
get_win_topics = [lda_model.get_document_topics(item, minimum_probability=0) for item in bow_win_corpus]

In [ ]:
get_win_topics

In [ ]:
import pandas as pd

In [ ]:
df_win = pd.DataFrame([[y[1] for y in x] for x in get_win_topics])

In [ ]:
df_win.columns=["Topic " + str(i) for i in range(10)]

In [ ]:
win_text=win_text.reset_index()

In [ ]:
df_win['facts'] = win_text['facts']

In [ ]:
df_win

In [ ]:
df_win.to_csv("Win_topics.csv")

In [ ]:
avg_prob_win=[]
for col in df_win.columns[0:10]:
    avg_prob_win.append(col + ": " + str(df_win[col].mean()))

In [ ]:
avg_prob_win

In [ ]:
get_lose_topics = [lda_model.get_document_topics(item, minimum_probability=0) for item in bow_lose_corpus]

In [ ]:
df_lose = pd.DataFrame([[y[1] for y in x] for x in get_lose_topics])

In [ ]:
df_lose.columns=["Topic " + str(i) for i in range(10)]

In [ ]:
lose_text=lose_text.reset_index()

In [ ]:
df_lose['facts']=lose_text['facts']

In [ ]:
df_lose

In [ ]:
df_lose.to_csv("Lose_topics.csv")

In [ ]:
avg_probs_lose=[]
for col in df_lose.columns[0:10]:
    avg_probs_lose.append(col + ": " + str((df_lose[col]).mean()))

In [ ]:
avg_probs_lose